In [1]:
import os
import sys
import random
import torch
import time
from pathlib import Path
from torch.optim import Adam
from torch.utils.data import DataLoader, Subset

c:\Users\42190\.conda\envs\LB\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sys.path.append("..")

In [3]:
from carca.data import CARCADataset, load_attrs, load_ctx, load_profiles
from carca.model import CARCA
from carca.train import train, evaluate

In [7]:
dataname = "train_dedoles.cz_2"
attrs = load_attrs(dataname)
ctx = load_ctx(dataname)
user_ids, item_ids, profiles = load_profiles(dataname)

In [8]:
n_items = attrs.shape[0]
n_ctx = next(iter(ctx.values())).shape[0]
n_attrs = attrs.shape[1]

In [4]:
# Hyper-parameters
learning_rate =  0.001
seq_len = 50
n_blocks = 1
n_heads = 3
dropout_rate = 0.5
l2_reg = 0.0
d_dim = 30
g_dim = 150
residual_sa = True
residual_ca = True
epochs = 800
batch_size = 128
beta1 = 0.9
beta2 = 0.98

In [10]:
train_data = CARCADataset(
    user_ids=user_ids,
    item_ids=item_ids,
    profiles=profiles,
    attrs=attrs,
    ctx=ctx,
    profile_seq_len=seq_len,
    target_seq_len=100,
    mode="train"
)
val_data = CARCADataset(
    user_ids=user_ids,
    item_ids=item_ids,
    profiles=profiles,
    attrs=attrs,
    ctx=ctx,
    profile_seq_len=seq_len,
    target_seq_len=100,
    mode="val"
)
# test_data = CARCADataset(
#     user_ids=user_ids,
#     item_ids=item_ids,
#     profiles=profiles,
#     attrs=attrs,
#     ctx=ctx,
#     profile_seq_len=seq_len,
#     target_seq_len=100,
#     mode="test"
# )

val_idx = random.sample(range(len(val_data)), 10_000) if len(val_data) > 10_000 else range(len(val_data))
val_sub = Subset(val_data, val_idx)
# test_idx = random.sample(range(len(test_data)), 10_000) if len(test_data) > 10_000 else range(len(test_data))
# test_sub = Subset(test_data, test_idx)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=8)
val_loader = DataLoader(val_sub, batch_size=batch_size, shuffle=False, num_workers=8)
# test_loader = DataLoader(test_sub, batch_size=batch_size, shuffle=False, num_workers=8)

In [11]:
model = CARCA(
    n_items=n_items,
    d=d_dim,
    g=g_dim,
    n_ctx=n_ctx,
    n_attrs=n_attrs,
    H=n_heads,
    p=dropout_rate,
    B=n_blocks,
    res_sa=residual_sa,
    res_ca=residual_ca
)

In [12]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cuda"
print(f"Using {device} device")
model = model.to(device)

Using cuda device


In [13]:
optim = Adam(model.parameters(), lr=learning_rate, weight_decay=l2_reg, betas=(beta1, beta2))

In [ ]:
model = train(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    optim=optim,
    epochs=epochs,
    early_stop=20,
    checkpoint=f"./{dataname}"
    # scheduler=scheduler
)

In [5]:
def train_loop():
    pnums = [(i * 4) + 3 for i in range(4, 12)]
    for pnum in pnums:
        start = time.time()

        dataname = f"train_urbanstore.cz_{pnum}"
        attrs = load_attrs(dataname)
        ctx = load_ctx(dataname)
        user_ids, item_ids, profiles = load_profiles(dataname)

        n_items = attrs.shape[0]
        n_ctx = next(iter(ctx.values())).shape[0]
        n_attrs = attrs.shape[1]

        train_data = CARCADataset(user_ids=user_ids, item_ids=item_ids, profiles=profiles, attrs=attrs,
                                ctx=ctx, profile_seq_len=seq_len, target_seq_len=100, mode="train")
        val_data = CARCADataset(user_ids=user_ids, item_ids=item_ids,profiles=profiles, attrs=attrs,
                                ctx=ctx, profile_seq_len=seq_len, target_seq_len=100, mode="val")

        val_idx = random.sample(range(len(val_data)), 10_000) if len(val_data) > 10_000 else range(len(val_data))
        val_sub = Subset(val_data, val_idx)

        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=8)
        val_loader = DataLoader(val_sub, batch_size=batch_size, shuffle=False, num_workers=8)

        model = CARCA(n_items=n_items, d=d_dim, g=g_dim, n_ctx=n_ctx, n_attrs=n_attrs, H=n_heads,
                      p=dropout_rate, B=n_blocks, res_sa=residual_sa, res_ca=residual_ca)
        
        device = "cuda"
        model = model.to(device)
        optim = Adam(model.parameters(), lr=learning_rate, weight_decay=l2_reg, betas=(beta1, beta2))

        model = train(model=model, train_loader=train_loader, val_loader=val_loader, device=device,
                      optim=optim, epochs=epochs, early_stop=50, checkpoint=f"./{dataname}")
        
        elapesd = time.time() - start
        Path(f"./{dataname}/{elapesd:.2f}.time").touch()

In [6]:
train_loop()

09:33:33 - Epoch 001: Loss = 0.6825
09:33:44 - Epoch 001: Loss = 0.8329 HR = 0.2901, NDCG = 0.1438
09:33:57 - Epoch 002: Loss = 0.6717
09:34:09 - Epoch 002: Loss = 0.7073 HR = 0.2753, NDCG = 0.1410
09:34:23 - Epoch 003: Loss = 0.6623
09:34:34 - Epoch 003: Loss = 0.7121 HR = 0.3262, NDCG = 0.1631
09:34:47 - Epoch 004: Loss = 0.5927
09:34:58 - Epoch 004: Loss = 0.4913 HR = 0.5446, NDCG = 0.3255
09:35:11 - Epoch 005: Loss = 0.4839
09:35:21 - Epoch 005: Loss = 0.5011 HR = 0.5321, NDCG = 0.3213
09:35:32 - Epoch 006: Loss = 0.4470
09:35:42 - Epoch 006: Loss = 0.3866 HR = 0.5290, NDCG = 0.3120
09:35:53 - Epoch 007: Loss = 0.4103
09:36:03 - Epoch 007: Loss = 0.4479 HR = 0.5312, NDCG = 0.3144
09:36:14 - Epoch 008: Loss = 0.3925
09:36:23 - Epoch 008: Loss = 0.6151 HR = 0.5495, NDCG = 0.3248
09:36:35 - Epoch 009: Loss = 0.3774
09:36:44 - Epoch 009: Loss = 0.3185 HR = 0.4934, NDCG = 0.2880
09:36:55 - Epoch 010: Loss = 0.3758
09:37:05 - Epoch 010: Loss = 0.6651 HR = 0.5473, NDCG = 0.3258
09:37:17 -